## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-30-22-36-49 +0000,nyt,New York’s Very Rich Reel Over the NYC Mayoral...,https://www.nytimes.com/2025/08/30/style/hampt...
1,2025-08-30-22-34-44 +0000,nyt,"Over 15 Killed in Gaza City, One Day After Isr...",https://www.nytimes.com/2025/08/30/world/middl...
2,2025-08-30-22-32-27 +0000,startribune,State Fair main entrance reopens after tempora...,https://www.startribune.com/state-fair-main-en...
3,2025-08-30-22-31-28 +0000,nypost,Disgraced priest who had sex on church altar g...,https://nypost.com/2025/08/30/us-news/ex-pries...
4,2025-08-30-22-27-10 +0000,missionlocal,"In break from past practice, 6 asylum-seekers ...",https://missionlocal.org/2025/08/in-break-from...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2351/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
73,trump,32
10,killed,10
140,israeli,9
74,tariffs,9
265,court,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
148,2025-08-30-04-15-00 +0000,wsj,"A federal judge in Washington, D.C., temporari...",https://www.wsj.com/politics/policy/judge-bloc...,116
178,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...,89
36,2025-08-30-18-46-00 +0000,wsj,The Trump administration is drawing up plans t...,https://www.wsj.com/politics/national-security...,70
171,2025-08-30-00-18-40 +0000,wapo,Appeals court strikes down Trump’s tariffs as ...,https://www.washingtonpost.com/business/2025/0...,62
90,2025-08-30-13-12-57 +0000,wapo,An appeals court said Trump couldn’t levy his ...,https://www.washingtonpost.com/business/2025/0...,58


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
148,116,2025-08-30-04-15-00 +0000,wsj,"A federal judge in Washington, D.C., temporari...",https://www.wsj.com/politics/policy/judge-bloc...
1,43,2025-08-30-22-34-44 +0000,nyt,"Over 15 Killed in Gaza City, One Day After Isr...",https://www.nytimes.com/2025/08/30/world/middl...
34,41,2025-08-30-19-00-34 +0000,nypost,Graphic video shows Israeli father throw himse...,https://nypost.com/2025/08/30/world-news/video...
178,36,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...
36,32,2025-08-30-18-46-00 +0000,wsj,The Trump administration is drawing up plans t...,https://www.wsj.com/politics/national-security...
163,31,2025-08-30-01-00-00 +0000,wsj,Minnesota Gov. Tim Walz campaigned on the nati...,https://www.wsj.com/politics/elections/we-aske...
23,30,2025-08-30-19-54-53 +0000,nypost,Twisted T-shirts pay tribute to Minneapolis ch...,https://nypost.com/2025/08/30/us-news/website-...
139,27,2025-08-30-07-22-55 +0000,bbc,US blocks Palestinian leader from attending UN...,https://www.bbc.com/news/articles/cjdym32z9v7o...
5,25,2025-08-30-22-23-30 +0000,nypost,"NYC shooting vics were ‘targeted,’ police sour...",https://nypost.com/2025/08/30/us-news/nyc-shoo...
47,25,2025-08-30-17-25-31 +0000,nypost,Russia launches sweeping attack on Southern Uk...,https://nypost.com/2025/08/30/world-news/russi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
